In [37]:
import folium
import geocoder
import requests
import numpy as np
import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans

## read html into data frame

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = requests.get(url).content
df_list = pd.read_html(html)
df_toronto = df_list[0]
df_toronto = df_toronto[df_toronto['Neighbourhood'] != 'Not assigned'].reset_index(drop=True)
df_toronto

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
...,...,...,...
204,M8Z,Etobicoke,Kingsway Park South West
205,M8Z,Etobicoke,Mimico NW
206,M8Z,Etobicoke,The Queensway West
207,M8Z,Etobicoke,Royal York South West


In [3]:
df_toronto.shape

(209, 3)

## Add lat and lng columns

In [4]:
lat_lng_coords = None
postal_code = 'M5G'

while(lat_lng_coords is None):
    g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]
latitude, longitude

KeyboardInterrupt: 

### geocoder lasts forever, use csv file instead

In [5]:
toronto_coords = pd.read_csv("https://cocl.us/Geospatial_data")
toronto_coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [6]:
df_toronto = df_toronto.join(toronto_coords.set_index('Postal Code'), on='Postcode')
df_toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Heights,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,43.718518,-79.464763


### save to csv file so I can continue processing on my local machine...

In [7]:
df_toronto.to_csv("./toronto_neighbours.csv", index=False)

## cluster

In [ ]:
# continue working on local machine, read data from csv
df_toronto = pd.read_csv("./toronto_neighbours.csv")
df_toronto.head()

### take a peek at the toronto map

In [8]:
def mark_circle(my_map, borough, neighborhood, lat, lng):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False) \
    .add_to(my_map)

In [9]:
address = 'Toronto'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
[mark_circle(map_toronto, row[1], row[2], row[3], row[4]) for row in df_toronto.values]
map_toronto

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### explore neighborhoods in toronto

In [10]:
# Foursquare credential
CLIENT_ID = 'B2OJ3YOLBYF35FVDQ5Y5A3PRA45JMAAZL1SL5N2D5PLBNU3J'
CLIENT_SECRET = 'CINVOH4OC0YGHMZD1FACIWKGD2UNL4Y4AQ4TX132TOA4MY4Q'
VERSION = '20180605'

In [11]:
def get_nearby_venues(names, latitudes, longitudes, radius=500, LIMIT=1000):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
#         print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [
        'Neighbourhood',
        'Neighbourhood Latitude',
        'Neighbourhood Longitude',
        'Venue',
        'Venue Latitude',
        'Venue Longitude',
        'Venue Category']
    return(nearby_venues)

In [12]:
toronto_venues = get_nearby_venues(
    names=df_toronto['Neighbourhood'],
    latitudes=df_toronto['Latitude'],
    longitudes=df_toronto['Longitude'])
print(toronto_venues.shape)
toronto_venues.head()

(4334, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [14]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Adelaide,100,100,100,100,100,100
Agincourt,5,5,5,5,5,5
Agincourt North,3,3,3,3,3,3
Albion Gardens,8,8,8,8,8,8
Alderwood,9,9,9,9,9,9
...,...,...,...,...,...,...
Woodbine Heights,9,9,9,9,9,9
York Mills,1,1,1,1,1,1
York Mills West,4,4,4,4,4,4


In [15]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 271 uniques categories.


### analyze each neighbour

In [16]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Social Club,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [20]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Social Club,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Adelaide,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.00,0.000000,

In [21]:
toronto_grouped.shape

(205, 272)

In [22]:
num_top_venues = 5
for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.05
2       Steakhouse  0.04
3  Thai Restaurant  0.04
4           Bakery  0.03


----Agincourt----
                       venue  freq
0                     Lounge   0.2
1             Breakfast Spot   0.2
2  Latin American Restaurant   0.2
3               Skating Rink   0.2
4             Clothing Store   0.2


----Agincourt North----
                 venue  freq
0                 Park  0.33
1           Playground  0.33
2  Arts & Crafts Store  0.33
3    Accessories Store  0.00
4   Mexican Restaurant  0.00


----Albion Gardens----
                 venue  freq
0             Pharmacy  0.12
1          Video Store  0.12
2           Beer Store  0.12
3       Sandwich Place  0.12
4  Fried Chicken Joint  0.12


----Alderwood----
          venue  freq
0   Pizza Place  0.22
1           Gym  0.11
2  Skating Rink  0.11
3  Dance Studio  0.11
4   Coffee Shop  0.11


----Bathurst Manor----
                venue  freq

                             venue  freq
0           Furniture / Home Store   0.5
1                   Baseball Field   0.5
2               Mexican Restaurant   0.0
3  Molecular Gastronomy Restaurant   0.0
4       Modern European Restaurant   0.0


----Eringate----
          venue  freq
0   Pizza Place  0.14
1   Coffee Shop  0.14
2  Liquor Store  0.14
3    Beer Store  0.14
4          Café  0.14


----Exhibition Place----
            venue  freq
0  Breakfast Spot  0.09
1            Café  0.09
2     Coffee Shop  0.09
3             Gym  0.05
4          Bakery  0.05


----Fairview----
                  venue  freq
0        Clothing Store  0.15
1           Coffee Shop  0.08
2  Fast Food Restaurant  0.08
3         Women's Store  0.05
4              Tea Room  0.05


----First Canadian Place----
         venue  freq
0  Coffee Shop  0.09
1         Café  0.07
2        Hotel  0.04
3   Restaurant  0.04
4   Steakhouse  0.04


----Flemingdon Park----
                  venue  freq
0                   

          venue  freq
0   Pizza Place  0.14
1   Coffee Shop  0.14
2  Liquor Store  0.14
3    Beer Store  0.14
4          Café  0.14


----Martin Grove----
                        venue  freq
0         Filipino Restaurant   0.5
1                  Print Shop   0.5
2           Accessories Store   0.0
3               Metro Station   0.0
4  Modern European Restaurant   0.0


----Martin Grove Gardens----
               venue  freq
0        Pizza Place  0.25
1               Park  0.25
2  Mobile Phone Shop  0.25
3           Bus Line  0.25
4  Accessories Store  0.00


----Maryvale----
                       venue  freq
0  Middle Eastern Restaurant   0.2
1                Auto Garage   0.2
2             Breakfast Spot   0.2
3                     Bakery   0.2
4             Sandwich Place   0.2


----Milliken----
                 venue  freq
0                 Park  0.33
1           Playground  0.33
2  Arts & Crafts Store  0.33
3    Accessories Store  0.00
4   Mexican Restaurant  0.00


----Mimico N

              venue  freq
0    Airport Lounge  0.13
1   Airport Service  0.13
2  Airport Terminal  0.13
3          Boutique  0.07
4           Airport  0.07


----South Steeles----
                 venue  freq
0             Pharmacy  0.12
1          Video Store  0.12
2           Beer Store  0.12
3       Sandwich Place  0.12
4  Fried Chicken Joint  0.12


----South of Bloor----
                  venue  freq
0                   Gym  0.07
1  Fast Food Restaurant  0.07
2           Wings Joint  0.07
3     Convenience Store  0.07
4           Social Club  0.07


----St. James Town----
         venue  freq
0  Coffee Shop  0.07
1         Café  0.06
2   Restaurant  0.06
3        Hotel  0.03
4       Bakery  0.03


----St. Phillips----
               venue  freq
0        Pizza Place  0.25
1               Park  0.25
2  Mobile Phone Shop  0.25
3           Bus Line  0.25
4  Accessories Store  0.00


----Steeles East----
                 venue  freq
0                 Park  0.33
1           Playground  

In [23]:
def most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [24]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Steakhouse,Thai Restaurant,Bakery,Bar,Restaurant,Salad Place,Sushi Restaurant,Asian Restaurant
1,Agincourt,Latin American Restaurant,Skating Rink,Lounge,Clothing Store,Breakfast Spot,Drugstore,Diner,Discount Store,Dog Run,Doner Restaurant
2,Agincourt North,Park,Arts & Crafts Store,Playground,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
3,Albion Gardens,Beer Store,Sandwich Place,Grocery Store,Fried Chicken Joint,Fast Food Restaurant,Video Store,Pharmacy,Pizza Place,Dim Sum Restaurant,Diner
4,Alderwood,Pizza Place,Gym,Coffee Shop,Skating Rink,Pharmacy,Pub,Dance Studio,Sandwich Place,Discount Store,Department Store


### clustering neighbors

In [27]:
# set number of clusters
k = 5

toronto_grouped_clustering = toronto_grouped.drop(columns=['Neighbourhood'])

# run k-means clustering
kmeans = KMeans(n_clusters=k, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 4, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [31]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,4.0,Park,Food & Drink Shop,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Hockey Arena,Coffee Shop,French Restaurant,Portuguese Restaurant,Yoga Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,1.0,Coffee Shop,Park,Pub,Bakery,Café,Mexican Restaurant,Theater,Restaurant,Breakfast Spot,French Restaurant
3,M6A,North York,Lawrence Heights,43.718518,-79.464763,1.0,Furniture / Home Store,Clothing Store,Accessories Store,Miscellaneous Shop,Carpet Store,Fraternity House,Vietnamese Restaurant,Coffee Shop,Boutique,Eastern European Restaurant
4,M6A,North York,Lawrence Manor,43.718518,-79.464763,1.0,Furniture / Home Store,Clothing Store,Accessories Store,Miscellaneous Shop,Carpet Store,Fraternity House,Vietnamese Restaurant,Coffee Shop,Boutique,Eastern European Restaurant


In [41]:
toronto_merged['Cluster Labels']

0      4.0
1      1.0
2      1.0
3      1.0
4      1.0
5      NaN
6      3.0
7      3.0
8      1.0
9      1.0
10     1.0
11     1.0
12     1.0
13     1.0
14     3.0
15     3.0
16     3.0
17     3.0
18     3.0
19     1.0
20     1.0
21     1.0
22     1.0
23     1.0
24     1.0
25     1.0
26     1.0
27     1.0
28     1.0
29     1.0
30     1.0
31     1.0
32     1.0
33     1.0
34     1.0
35     1.0
36     4.0
37     1.0
38     1.0
39     1.0
40     1.0
41     1.0
42     1.0
43     1.0
44     1.0
45     1.0
46     1.0
47     1.0
48     1.0
49     1.0
50     1.0
51     1.0
52     4.0
53     1.0
54     1.0
55     1.0
56     1.0
57     1.0
58     4.0
59     1.0
60     1.0
61     1.0
62     1.0
63     1.0
64     1.0
65     1.0
66     1.0
67     1.0
68     4.0
69     4.0
70     1.0
71     1.0
72     1.0
73     1.0
74     1.0
75     1.0
76     1.0
77     1.0
78     1.0
79     1.0
80     2.0
81     2.0
82     1.0
83     1.0
84     1.0
85     1.0
86     1.0
87     4.0
88     4.0
89     4.0
90     1.0

In [42]:
toronto_merged.iloc[5]

Postcode                               M9A
Borough                   Downtown Toronto
Neighbourhood                 Queen's Park
Latitude                           43.6679
Longitude                         -79.5322
Cluster Labels                         NaN
1st Most Common Venue                  NaN
2nd Most Common Venue                  NaN
3rd Most Common Venue                  NaN
4th Most Common Venue                  NaN
5th Most Common Venue                  NaN
6th Most Common Venue                  NaN
7th Most Common Venue                  NaN
8th Most Common Venue                  NaN
9th Most Common Venue                  NaN
10th Most Common Venue                 NaN
Name: 5, dtype: object

In [43]:
print(toronto_merged.shape)
toronto_merged.dropna(subset=['Cluster Labels'], inplace=True)
print(toronto_merged.shape)

(209, 16)
(207, 16)


In [48]:
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)
toronto_merged.dtypes

Postcode                   object
Borough                    object
Neighbourhood              object
Latitude                  float64
Longitude                 float64
Cluster Labels              int64
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object

In [49]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters